In [ ]:
from transformers import BertTokenizer

protein_tokenizer = BertTokenizer.from_pretrained("data/target/protein_tokenizer", do_lower_case=False)
vocab_size = len(protein_tokenizer.get_vocab().keys())

print(f"load tokenizer\nvocab size: {vocab_size}\nspecial tokens: {protein_tokenizer.all_special_tokens}")

In [ ]:
import os
import pickle

if not os.path.exists("data/target/X.pkl"):
    from sklearn.model_selection import train_test_split
    
    with open("data/target/target_sequences.pkl", 'rb') as f:
        data = pickle.load(f)
    
        print(f"load dataset ... # of data: {len(data)}")
    
    X_train, X_test = train_test_split(data, test_size=0.1, random_state=42, shuffle=True)
    X_train, X_valid = train_test_split(X_train, test_size=0.1, random_state=42, shuffle=True)
    
    with open("data/target/X.pkl", "wb") as f:
        pickle.dump([X_train, X_valid, X_test], f)
else:
    with open("data/target/X.pkl", "rb") as f:
        X_train, X_valid, X_test = pickle.load(f)
        
print(f"load dataset\nX_train: {len(X_train)}\nX_valid: {len(X_valid)}\nX_test: {len(X_test)}")

In [ ]:
import numpy as np
import seaborn as sns

sns.distplot(np.array([len(x) for x in X_train]))

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, RandomSampler
from transformers import DataCollatorForLanguageModeling

max_seq_len = 1024

class MaskedLMDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        
    def encode(self, data):
        return self.tokenizer.encode(" ".join(data), max_length=self.max_length, truncation=True)
        
        
    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, idx):
        return torch.tensor(self.encode(self.data[idx]), dtype=torch.long)
    
    
data_collator_train = DataCollatorForLanguageModeling(
    tokenizer=protein_tokenizer, mlm=True, mlm_probability=0.3
)

data_collator_valid = DataCollatorForLanguageModeling(
    tokenizer=protein_tokenizer, mlm=True, mlm_probability=0.15
)

train_dataset = MaskedLMDataset(X_train, protein_tokenizer, max_length=max_seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=6, collate_fn=data_collator_train, 
                              num_workers=3, pin_memory=False, prefetch_factor=4, 
                              shuffle=True)

valid_dataset = MaskedLMDataset(X_valid, protein_tokenizer, max_length=max_seq_len)
valid_dataloader = DataLoader(valid_dataset, batch_size=6, collate_fn=data_collator_valid, 
                              num_workers=3, pin_memory=False, prefetch_factor=4)

test_dataset = MaskedLMDataset(X_test, protein_tokenizer, max_length=max_seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=6, collate_fn=data_collator_valid, 
                             num_workers=3, pin_memory=False, prefetch_factor=4)


In [ ]:
import torchmetrics
import torchmetrics.functional as FM
import pytorch_lightning as pl
from transformers import BertForMaskedLM
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


class Bert(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
        for param in self.model.base_model.encoder.layer[0:-1].parameters():
            param.requires_grad = False
        
        
    def forward(self, input_ids, labels):
        return self.model(input_ids=input_ids, labels=labels)

       
    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        labels = batch['labels']
        
        output = self(input_ids, labels)

        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)
        
        self.log('train_loss', float(loss), on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_accuracy", FM.accuracy(preds[labels > 0], labels[labels > 0]), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    
    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        labels = batch['labels']
        
        output = self(input_ids, labels)

        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)
        
        self.log('valid_loss', float(loss), on_step=False, on_epoch=True, prog_bar=True)
        self.log("valid_accuracy", FM.accuracy(preds[labels > 0], labels[labels > 0]), on_step=False, on_epoch=True, prog_bar=True, logger=True)
    
    
    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        labels = batch['labels']
        
        output = self(input_ids, labels)

        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)
        
        self.log('test_loss', float(loss), on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_accuracy", FM.accuracy(preds[labels > 0], labels[labels > 0]), on_step=False, on_epoch=True, prog_bar=True, logger=True)
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)
    
        return {"optimizer": optimizer, "lr_scheduler": scheduler}
    
    
model = Bert()
callbacks = [
    ModelCheckpoint(monitor='valid_loss', save_top_k=5, dirpath='weights/protein_bert_pretraining_masking_rate_30', filename='protein_bert-{epoch:03d}-{valid_loss:.4f}-{valid_accuracy:.4f}'),
]

trainer = pl.Trainer(max_epochs=50, gpus=1, enable_progress_bar=True, callbacks=callbacks)

In [ ]:
trainer.fit(model, train_dataloader, valid_dataloader)

In [ ]:
ckpt_fname = "protein_bert-epoch=043-valid_loss=0.5893-valid_accuracy=0.8369.ckpt"
model = Bert().load_from_checkpoint("weights/protein_bert_pretraining_masking_rate_30/" + ckpt_fname)

trainer.test(model, test_dataloader)

In [ ]:
model.model.base_model.save_pretrained("weights/protein_bert")